In [1]:
import numpy as np

from Windowing import *
import torch
from torch.linalg import inv, det

# Introduction
Here we implement and test the GP method

# Method

In [2]:
win = Windowing(aggCurves)
win.set_points([16,   23,   32,   45,   64,   91,  128,  181,  256,  362,  512,
        724, 1024, 1448, 2048, 2896])

,Count
size_train,
16,245
23,246
32,246
40,1
45,244
...,...
1048576,2
1482910,1
1486391,1


In [5]:
win.segment.shape

(248, 20, 16)

In [8]:
win.train.shape

(248, 20, 15)

In [9]:
win.data.shape

(248, 20, 15, 15)

In [44]:
win.target.shape

(248, 20, 15)

In [45]:
win.data[0,0,:,0]

array([0.6917696,       nan,       nan,       nan,       nan,       nan,
             nan,       nan,       nan,       nan,       nan,       nan,
             nan,       nan,       nan])

In [48]:
win.target[0,0]

array([0.7354512, 0.7846216, 0.8307736, 0.8681464, 0.8978816, 0.9212488,
       0.9391992, 0.9485776, 0.9562008, 0.9610792, 0.9648808, 0.9666088,
       0.96946  , 0.9727528,       nan])

In [28]:
train_anchors = np.array(win.train_anchors)


In [42]:
cov = cov_kernel(train_anchors, train_anchors[...,None], np.array([1,1,1]))
cov.shape

(15, 15)

## Functions for GP
- Mean function (actually not required as we have a constant mean)
- Covariance kernel
- Log marginal likelihood for optimization of hyperparameters for GP
- Prediction function

In [429]:
# def mean_func(X, param):
#     '''
#     We have a constant mean function, so it doesnt matter what the input is
#
#     :param param: constant mean used
#     :return: mean function output at X
#     '''
#
#     return param

def cov_kernel(X1,X2, params):
    '''
    Exponential decay kernel plus additive noise used in Freeze-Thaw for learning curves

    :param X1: Training anchors input 1
    :param X2: Training anchors input 2
    :param params: [alpha, beta, sigma_squared] parameters for the kernel
    :return: covariance matrix output at X1,X2
    '''
    alpha, beta, sigma = params[...,None,None]
    exp_decay = beta**alpha / (X1 + X2 + beta)**alpha
    noise = sigma**2 * (X1 == X2)
    return exp_decay + noise



In [491]:
def log_marginal_likelihood(params, y, X):
    cov = cov_kernel(X, X[...,None],params)
    inv_cov = inv(cov)
    matmul = torch.einsum('...i,...ij,...j', y, inv_cov, y)
    det_cov = det(cov)
    n = len(X)
    res = -1/2 * matmul
    return res

In [467]:
X

tensor([16., 23., 32.])

In [492]:
lr = 0.01
y = 1- torch.tensor(win.data[0,0,:,-1], dtype=torch.float32)
X = torch.tensor(win.train_anchors, dtype=torch.float32)
params = torch.tensor([0.5,0.5,0.5], dtype=torch.float32).requires_grad_(True)

optimizer = torch.optim.AdamW([params], lr=lr)

prob = []
params_list = []
for i in range(1000):
    optimizer.zero_grad()
    loss = -log_marginal_likelihood(params, y, X)
    prob.append(np.exp(-loss.item()))
    params_list.append(params.clone().detach().numpy())
    loss.backward()
    optimizer.step()
    if i % 100 == 0:
        print(loss)

tensor(0.1950, grad_fn=<NegBackward0>)
tensor(0.0428, grad_fn=<NegBackward0>)
tensor(0.0289, grad_fn=<NegBackward0>)
tensor(0.0224, grad_fn=<NegBackward0>)
tensor(0.0185, grad_fn=<NegBackward0>)
tensor(0.0159, grad_fn=<NegBackward0>)
tensor(0.0140, grad_fn=<NegBackward0>)
tensor(0.0126, grad_fn=<NegBackward0>)
tensor(0.0115, grad_fn=<NegBackward0>)
tensor(0.0105, grad_fn=<NegBackward0>)


In [480]:
torch.tensor(params_list[11])
cov = cov_kernel(X, X[...,None],torch.tensor(params_list[-1]))
inv_cov = inv(cov)
matmul = torch.einsum('...i,...ij,...j', y, inv_cov, y)
det_cov = det(cov)
n = len(X)
res = -1/2 * matmul -1/2 * torch.log(det_cov) - n/2 * torch.log(torch.tensor(2*torch.pi))
res2 = 1/((2*np.pi)**n * det_cov) * torch.exp(-1/2 * matmul)

In [493]:
params_list

[array([0.5, 0.5, 0.5], dtype=float32),
 array([0.48995, 0.50995, 0.50995], dtype=float32),
 array([0.47992206, 0.5198651 , 0.5198769 ], dtype=float32),
 array([0.46992996, 0.5297236 , 0.52976733], dtype=float32),
 array([0.45998734, 0.5395044 , 0.53960866], dtype=float32),
 array([0.4501074 , 0.54918766, 0.5493895 ], dtype=float32),
 array([0.44030297, 0.5587546 , 0.5590994 ], dtype=float32),
 array([0.4305864 , 0.56818795, 0.56872904], dtype=float32),
 array([0.4209695, 0.5774718, 0.5782702], dtype=float32),
 array([0.41146344, 0.5865916 , 0.5877157 ], dtype=float32),
 array([0.40207884, 0.59553456, 0.59705937], dtype=float32),
 array([0.3928256 , 0.60428935, 0.606296  ], dtype=float32),
 array([0.38371295, 0.61284626, 0.6154214 ], dtype=float32),
 array([0.37474933, 0.621197  , 0.62443215], dtype=float32),
 array([0.36594254, 0.6293348 , 0.6333257 ], dtype=float32),
 array([0.35729966, 0.63725436, 0.6421003 ], dtype=float32),
 array([0.348827  , 0.64495164, 0.6507548 ], dtype=float3

In [497]:
def log_marginal_likelihood(params, y, X):
    '''
    Log marginal likelihood for optimization of hyperparameters for GP

    :param params: arameters for the covariance kernel and mean function
    :param y: actual outputs
    :param X: training anchors
    :return: log marginal likelihood
    '''
    # # set mean
    # means = params[0][...,None,:]

    # make nans 0
    # y = y - means
    nan = torch.isnan(y)
    nanT = nan.transpose(-1,-2)
    nan_ind = torch.where(nanT)
    y = torch.where(nan, torch.tensor(0.0), y)
    yT = y.transpose(-1,-2)

    # get covariance matrix and set rows/columns where nan to 0 as these are technically not datapoints
    cov = cov_kernel(X, X[...,None],params)
    cov[nanT,:] = 0
    cov.transpose(-1,-2)[nanT,:] = 0
    cov[nan_ind[0],nan_ind[1],nan_ind[2],nan_ind[3],nan_ind[3]] = 1 # The trick is to set the "fake" datapoints diagonal to 1 and rest of its row and column to 1, this way it is like it's not there for the determinant and inverse
    inv_cov = inv(cov)

    # calculate the matrix multiplication
    matmul = torch.einsum('...i,...ij,...j', yT, inv_cov, yT)

    res = -1/2 * matmul
    return res




## Torch optimisation loop

In [500]:
def optimize_params(params, y, X, lr = 0.001, n_iter = 100):
    '''
    Optimise the hyperparameters for the GP

    :param params: initial parameters for the covariance kernel and mean function
    :param y: actual outputs
    :param X: training anchors
    :param lr: learning rate
    :param n_iter: number of iterations
    :return: optimised parameters
    '''
    params = params.clone().requires_grad_(True)
    optimizer = torch.optim.AdamW([params], lr=lr)
    gr = torch.ones(params.shape[1:])

    for i in range(n_iter):
        optimizer.zero_grad()
        loss = -log_marginal_likelihood(params, y, X)
        loss.backward(gradient=gr)
        optimizer.step()
        print(f'Iteration {i} marginal likelihood: {np.exp(-loss.mean().item())}')
    return params

In [501]:
y = torch.tensor(win.data, dtype=torch.float32)
X = torch.tensor(win.train_anchors, dtype=torch.float32)
params = torch.tensor([0.5,0.5, 0.5], dtype=torch.float32)
shp = np.hstack((np.array(y.shape[:2]), np.array(y.shape[3])))
params = params.repeat(*shp, 1)

# params at front
params = params.permute(*np.roll(np.arange(len(params.shape)),1))

params = optimize_params(params, y, X, lr = 0.01, n_iter = 100)

Iteration 0 marginal likelihood: 0.09562672826246682
Iteration 1 marginal likelihood: 0.10890091292583436
Iteration 2 marginal likelihood: 0.12298937018150734
Iteration 3 marginal likelihood: 0.13781627580639588
Iteration 4 marginal likelihood: 0.15329814070044503
Iteration 5 marginal likelihood: 0.16934654593644333
Iteration 6 marginal likelihood: 0.18587037440942183
Iteration 7 marginal likelihood: 0.20277830898878801
Iteration 8 marginal likelihood: 0.21998056764732424
Iteration 9 marginal likelihood: 0.23739060723513647
Iteration 10 marginal likelihood: 0.25492612717174873
Iteration 11 marginal likelihood: 0.27251055949434594
Iteration 12 marginal likelihood: 0.29007308217312056
Iteration 13 marginal likelihood: 0.307549573253028
Iteration 14 marginal likelihood: 0.3248824362140888
Iteration 15 marginal likelihood: 0.34202084073414385
Iteration 16 marginal likelihood: 0.3589203626876506
Iteration 17 marginal likelihood: 0.37554282987178245
Iteration 18 marginal likelihood: 0.391856

In [503]:
def predict(params, X, y, X_star):
    '''
    Predict the output for a given set of inputs

    :param params: parameters for the covariance kernel and mean function
    :param X: inputs
    :return: predicted output
    '''

    nan = torch.isnan(y)
    nanT = nan.transpose(-1,-2)
    nan_ind = torch.where(nanT)
    y = torch.where(nan, torch.tensor(0.0), y)
    yT = y.transpose(-1,-2)

    # get covariance matrix and set rows/columns where nan to 0 as these are technically not datapoints
    cov = cov_kernel(X, X[...,None],params)
    cov[nanT,:] = 0
    cov.transpose(-1,-2)[nanT,:] = 0
    cov[nan_ind[0],nan_ind[1],nan_ind[2],nan_ind[3],nan_ind[3]] = 1 # The trick is to set the "fake" datapoints diagonal to 1 and rest of its row and column to 1, this way it is like it's not there for the determinant and inverse
    inv_cov = inv(cov)

    # Get cov star
    cov_star = cov_kernel(X, X_star[...,None],params)
    cov_star[nanT,:] = 0

    return res

tensor([-0.0389,  0.8103,  0.8129], grad_fn=<SelectBackward0>)

In [530]:
y = torch.tensor(win.data, dtype=torch.float32)
X = torch.tensor(win.train_anchors, dtype=torch.float32)
X_star = torch.tensor(win.target_anchors[1:], dtype=torch.float32)

nan = torch.isnan(y)
nanT = nan.transpose(-1,-2)
nan_ind = torch.where(nanT)
y = torch.where(nan, torch.tensor(0.0), y)

# get covariance matrix and set rows/columns where nan to 0 as these are technically not datapoints
cov = cov_kernel(X, X[...,None],params)
cov[nanT,:] = 0
cov.transpose(-1,-2)[nanT,:] = 0
cov[nan_ind[0],nan_ind[1],nan_ind[2],nan_ind[3],nan_ind[3]] = 1 # The trick is to set the "fake" datapoints diagonal to 1 and rest of its row and column to 1, this way it is like it's not there for the determinant and inverse

inv_cov = inv(cov)

# Get cov star
cov_star = cov_kernel(X, X_star[...,None],params)

# einsum
res = torch.einsum('...ij,...jk,...k', cov_star, inv_cov, y)


In [538]:
y[0,0,:,-1]

tensor([0.6918, 0.7355, 0.7846, 0.8308, 0.8681, 0.8979, 0.9212, 0.9392, 0.9486,
        0.9562, 0.9611, 0.9649, 0.9666, 0.9695, 0.0000])

In [541]:
res[0,0,-1,-3]

tensor(2.3192e-07, grad_fn=<SelectBackward0>)

In [529]:
inv_cov.shape

torch.Size([248, 20, 15, 15, 15])

In [528]:
cov_star.shape

torch.Size([248, 20, 15, 14, 15])

In [517]:
cov_star.shape

torch.Size([248, 20, 15, 14, 15])

In [519]:
y.shape

torch.Size([248, 20, 15, 15])

In [512]:
params[:,0,0,0]

tensor([-0.2294,  0.8053,  0.9536], grad_fn=<SelectBackward0>)

## This is where I tested

In [206]:
# set mean
mean = 0.5

# actual outputs + make nans 0 for einsum
y = torch.tensor(win.data, dtype=torch.float32) - mean
nan = torch.isnan(y)
nanT = nan.transpose(-1,-2)
nan_ind = torch.where(nanT)
y = torch.where(nan, torch.tensor(0.0), y)

# Train acnhors
train_anchors = torch.tensor(win.train_anchors, dtype=torch.float32)

# set hyperparameters
params = torch.tensor([0.5, 1, 1], dtype=torch.float32)
shp = np.hstack((np.array(y.shape[:2]), np.array(y.shape[3])))
params = params.repeat(*shp, 1)

# params at front
params = params.permute(*np.roll(np.arange(len(params.shape)),1))

# get covariance matrix and set rows/columns where nan to 0 as these are technically not datapoints
cov = cov_kernel(train_anchors, train_anchors[...,None],params)
cov[nanT,:] = 0
cov.transpose(-1,-2)[nanT,:] = 0
cov[nan_ind[0],nan_ind[1],nan_ind[2],nan_ind[3],nan_ind[3]] = 1 # The trick is to set the "fake" datapoints diagonal to 1 and rest of its row and column to 1, this way it is like it's not there for the determinant and inverse
inv_cov = inv(cov)

# calculate the matrix multiplication
matmul = torch.einsum('...i,...ij,...j', y.transpose(-1,-2), inv_cov, y.transpose(-1,-2))

# we also need the determinant of the covariance matrix
det_cov = det(cov)

# and the number of actual datapoints
n = len(train_anchors) - nan.sum(axis = -2)

res = -1/2 * matmul -1/2 * torch.log(det_cov) - n/2 * torch.log(torch.tensor(2*torch.pi))

In [207]:
matmul.shape

torch.Size([248, 20, 15])

In [208]:
res

tensor([[[ -1.0148,  -2.0120,  -3.0026,  ..., -12.9652, -13.9693, -14.9756],
         [ -1.0133,  -2.0098,  -2.9994,  ..., -12.9744, -13.9847, -14.9975],
         [ -1.0009,  -1.9849,  -2.9613,  ..., -12.8883, -13.8918, -14.8982],
         ...,
         [ -1.0076,  -1.9954,  -2.9713,  ..., -12.7774, -13.7783, -14.7834],
         [ -0.0000,  -0.0000,  -0.0000,  ...,  -0.0000,  -0.0000,  -0.0000],
         [ -0.0000,  -0.0000,  -0.0000,  ...,  -0.0000,  -0.0000,  -0.0000]],

        [[ -1.0456,  -2.0524,  -3.0361,  ..., -12.4359, -13.3636, -14.2900],
         [ -1.0495,  -2.0579,  -3.0409,  ..., -12.5937, -13.5734, -14.5581],
         [ -1.0724,  -2.1019,  -3.1068,  ..., -12.6270, -13.5963, -14.5725],
         ...,
         [ -1.0655,  -2.0895,  -3.0876,  ..., -12.5311, -13.4661, -14.4042],
         [ -1.0650,  -2.0771,  -3.0632,  ..., -12.5238, -13.4679, -14.4113],
         [ -0.0000,  -0.0000,  -0.0000,  ...,  -0.0000,  -0.0000,  -0.0000]],

        [[ -1.0407,  -2.0598,  -3.0617,  ...

In [202]:
res

tensor([[[ -1.0148,  -2.0120,  -3.0026,  ..., -12.9652, -13.9693, -14.9756],
         [ -1.0133,  -2.0098,  -2.9994,  ..., -12.9744, -13.9847, -14.9975],
         [ -1.0009,  -1.9849,  -2.9613,  ..., -12.8883, -13.8918, -14.8982],
         ...,
         [ -1.0076,  -1.9954,  -2.9713,  ..., -12.7774, -13.7783, -14.7834],
         [ -0.0000,  -0.0000,  -0.0000,  ...,  -0.0000,  -0.0000,  -0.0000],
         [ -0.0000,  -0.0000,  -0.0000,  ...,  -0.0000,  -0.0000,  -0.0000]],

        [[ -1.0456,  -2.0524,  -3.0361,  ..., -12.4359, -13.3636, -14.2900],
         [ -1.0495,  -2.0579,  -3.0409,  ..., -12.5937, -13.5734, -14.5581],
         [ -1.0724,  -2.1019,  -3.1068,  ..., -12.6270, -13.5963, -14.5725],
         ...,
         [ -1.0655,  -2.0895,  -3.0876,  ..., -12.5311, -13.4661, -14.4042],
         [ -1.0650,  -2.0771,  -3.0632,  ..., -12.5238, -13.4679, -14.4113],
         [ -0.0000,  -0.0000,  -0.0000,  ...,  -0.0000,  -0.0000,  -0.0000]],

        [[ -1.0407,  -2.0598,  -3.0617,  ...

In [133]:
inv_cov

array([[ 9.72042725e-01, -2.29787231e-02, -1.86854990e-02,
        -1.47030689e-02, -1.12007496e-02, -8.36022889e-03,
        -6.19869856e-03, -4.51977466e-03, -3.26511043e-03,
        -2.34366939e-03, -1.67402318e-03, -1.19202830e-03,
        -8.46708936e-04, -6.00623575e-04, -4.25527160e-04],
       [-2.29787231e-02,  9.80468894e-01, -1.63661872e-02,
        -1.32554962e-02, -1.03675265e-02, -7.91133384e-03,
        -5.96855316e-03, -4.41266745e-03, -3.22171857e-03,
        -2.33098479e-03, -1.67478566e-03, -1.19770773e-03,
        -8.53399597e-04, -6.06730024e-04, -4.30546218e-04],
       [-1.86854990e-02, -1.63661872e-02,  9.85891869e-01,
        -1.17584763e-02, -9.45181280e-03, -7.38741310e-03,
        -5.68277039e-03, -4.26894077e-03, -3.15589536e-03,
        -2.30515524e-03, -1.66804393e-03, -1.19915086e-03,
        -8.57699934e-04, -6.11474848e-04, -4.34776956e-04],
       [-1.47030689e-02, -1.32554962e-02, -1.17584763e-02,
         9.89897210e-01, -8.37307471e-03, -6.73054061

In [82]:
A = np.array([[1,2,3],[4,5,6],[1,2,-1]])
A = torch.tensor(A,dtype=torch.float32)
print(inv(A))

tensor([[-1.4167,  0.6667, -0.2500],
        [ 0.8333, -0.3333,  0.5000],
        [ 0.2500,  0.0000, -0.2500]])


In [138]:
B = torch.tensor(np.array(torch.clone(A)),dtype=torch.float32)
B[1,:] = 0
B[:,1] = 0
B[1,1] = 1
print(B)
print(inv(B))

tensor([[ 1.,  0.,  3.],
        [ 0.,  1.,  0.],
        [ 1.,  0., -1.]])
tensor([[ 0.2500,  0.0000,  0.7500],
        [ 0.0000,  1.0000,  0.0000],
        [ 0.2500,  0.0000, -0.2500]])


In [46]:
C = torch.tensor(np.array([[1,3],[1,-1]]),dtype=torch.float32)
print(C)
print(inv(C))

tensor([[ 1.,  3.],
        [ 1., -1.]])
tensor([[ 0.2500,  0.7500],
        [ 0.2500, -0.2500]])


In [86]:
Id = torch.eye(3)
Id[1,:] = 0
Id[:,1] = 0
print(Id)

tensor([[1., 0., 0.],
        [0., 0., 0.],
        [0., 0., 1.]])


In [89]:
torch.linalg.solve(B, Id)

_LinAlgError: torch.linalg.solve: The solver failed because the input matrix is singular.

In [79]:
inv_np(A[np.ix_([0,2], [0,2])])

array([[ 0.25,  0.75],
       [ 0.25, -0.25]], dtype=float32)

In [90]:
torch.linalg.pinv(B)

tensor([[ 0.2500,  0.0000,  0.7500],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.2500,  0.0000, -0.2500]])